*The dataset used in this example is copyrighted by West Coast Regional Council, LINZ (2021) and distributed by [OpenTopography](https://opentopography.org) under the [CC-BY-4.0](https://creativecommons.org/licenses/by/4.0/) license. [DOI](https://doi.org/10.5069/G9Z31WV0)*

## Mapping segmentations to filter pipelines

When talking about *adaptive* ground point filtering in AFwizard we have two types of adaptivity in mind: Parameter adaptivity and spatial adaptivity. This notebook describes the details of how spatial adaptivity is implemented in AFwizard. It assumes that you have already created a suitable segmentation of your dataset into spatial features (e.g. in a GIS). We will then see how we can attach filter pipeline information to that segmentation file.

In [ ]:
import afwizard

We again work on our demonstrator dataset:

In [ ]:
ds = afwizard.DataSet(filename="500k_NZ20_Westport.laz", spatial_reference="EPSG:2193")

Next, we import the segmentation the GeoJSON file. It is assumed to contain a `FeatureCollection` in the sense of the GeoJSON standard where each features combines the geometric information of the segment (`Polygon` or `Multipolygon`) and a number of properties. One of these properties should contain your custom classification of the segments into classes.

In [ ]:
segmentation = afwizard.load_segmentation(
    "westportSegmentation.geojson", spatial_reference="EPSG:4326"
)

As we are trying to map features to filter pipelines, we also need to load some filter pipelines. Here, we are directly opening these using `load_filter`. In practice, these should be selected from available filter libraries using e.g. the tools described in [Working with filter libraries](libraries.ipynb).

In [ ]:
pipelines = [
    afwizard.load_filter("testfilter1.json"),
    afwizard.load_filter("testfilter2.json"),
]

The core task of assigning filter piplines is done by the `assign_pipeline` function which allows us to interactively set filter pipelines. On the right hand side, we can choose which property of our GeoJSON file contains the classification information. A feature can be highlighted on the map by clicking the <i class="fas fa-map-marker-alt"></i> button. For each class of segments, a pipeline can be selected from the dropdown menu:

In [ ]:
assigned_segmentation = afwizard.assign_pipeline(
    ds, segmentation=segmentation, pipelines=pipelines
)

Once the pipelines are assigned, the returned segmentation object has a new property called "pipeline" that will direct the adaptivefiltering command line interface to the corresponding filter pipeline file. The modified file can be saved to disk by using the `save` method:

In [ ]:
assigned_segmentation.save("assigned_segmentation.geojson")

It is worth noting that `adaptivefiltering` does not store the entire filter configuration in the GeoJSON object. This is done to allow further refinement of the used filter pipelines after the segmentation is created. Also, it does not store absolute or relative paths to your filter pipelines, because these could always change when moving to new hardware or when reorganizing your project. Instead it stores the metadata of your filter (in hashed form) and compares it against the metadata of the filter pipelines in your currently loaded filter libraries. If metadata is ambiguous across the given filter libraries, an error will be thrown.